<a href="https://colab.research.google.com/github/vandana-iyer/RecSys-Instacart/blob/master/InstacartMarketBasketAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instacart market basket analysis

In [1]:
# Authenticate and download datasets from Kaggle

!mkdir -p ~/.kaggle
!wget -O ~/.kaggle/kaggle.json https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

!pip install --upgrade kaggle

!kaggle competitions download -c instacart-market-basket-analysis
!unzip instacart-market-basket-analysis.zip && rm instacart-market-basket-analysis.zip

# unzip all csv files
!unzip \*.csv.zip
# remove unwanted files/directories
!rm -rf __MACOSX
# !rm *.zip

--2020-04-21 03:00:47--  https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b6g4cawc664i4u9/kaggle.json [following]
--2020-04-21 03:00:48--  https://www.dropbox.com/s/raw/b6g4cawc664i4u9/kaggle.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc972656eac50752a8d6bafc5531.dl.dropboxusercontent.com/cd/0/inline/A2P67njKdTMgQsKt8XVF-4L5lHlD4dgCegVZWgT9PQ8gphrmU33lXD4TonFFJe6f8vZHPhypIxb5vaPJiAKYP0hbfSzsU-IexEiqSBMptGrXv0v65Q00R3qVlZ8rGCJX-2s/file# [following]
--2020-04-21 03:00:48--  https://uc972656eac50752a8d6bafc5531.dl.dropboxusercontent.com/cd/0/inline/A2P67njKdTMgQsKt8XVF-4L5lHlD4dgCegVZWgT9PQ8gphrmU33lXD4TonFFJe6f8vZHPhypIxb5vaPJiAKYP0hbfSzsU-IexEiqSBMptGrXv0v6

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
order_products = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')

In [4]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


**Is product to department many-to many?**

As observed below, since there are 49688 unique products that belong to 21 departments (no null departments), we can conclude that each product belongs to exactly one department, but each department has many products. So the relationship between department-to-product is one-to many.


In [11]:
print(f'Total unique products: {len(products["product_id"].unique())}')
print(f'Total unique departments: {len(products["department_id"].unique())}')

# Do products belong to any null departments?
products[products['department_id'].isnull()] 

# 

Total unique products: 49688
Total unique departments: 21


,product_id,product_name,aisle_id,department_id


**Is department to aisle one-to-many or many-to-many?**

As observed below, each aisle is associated with exactly one department and each department has many aisles. Hence the department-to-aisle is a one-to-many relationship

In [12]:
# Get total aisles in each department 
total_aisles_in_each_dept = products.groupby('department_id').agg({'aisle_id': pd.Series.nunique}).rename(columns={'aisle_id': 'aisle_count'}).sort_values(by='aisle_count', ascending=False)
# Is any department is associated with one or more aisles?
total_departments_in_each_aisle = products.groupby('aisle_id').agg({'department_id': pd.Series.nunique}).sort_values(by='department_id', ascending=False).rename(columns={'department_id': 'department_count'})

print(f'Maximum aisles in a particular department {total_aisles_in_each_dept[:1]["aisle_count"].values[0]}')
print(f'Maximum departments associated with each aisle {total_departments_in_each_aisle[:1]["department_count"].values[0]}')
products[products['aisle_id'].isnull()]

Maximum aisles in a particular department 17
Maximum departments associated with each aisle 1


,product_id,product_name,aisle_id,department_id


**Is aisle to product one-to-many or many-to-many?**

As we can observe below, every aisle has lot of products, but each product belongs to exactly one aisle. So aisle to products is one-to-many

In [13]:
# Get total products in each aisle 
max_products_in_each_aisle = products.groupby('aisle_id').agg({'product_id': pd.Series.nunique}).rename(columns={'product_id': 'product_count'}).sort_values(by='product_count', ascending=False)
# Is any aisle associated with one or more products
total_aisles_for_each_product = products.groupby('product_id').agg({'aisle_id': pd.Series.nunique}).rename(columns={'aisle_id': 'aisle_count'}).sort_values(by='aisle_count', ascending=False)

print(f'Maximum products in a each aisle {max_products_in_each_aisle[:1]["product_count"].values[0]}')
print(f'Maximum aisles associated with each product {total_aisles_for_each_product[:1]["aisle_count"].values[0]}')

Maximum products in a each aisle 1258
Maximum aisles associated with each product 1


In [14]:
# dept_id, no_of products 

products[['department_id', 'product_id']].groupby(['department_id']).count().merge(departments, on='department_id').sort_values('product_id', ascending=False).rename(columns={'product_id': 'product_count'})

,department_id,product_count,department
10,11,6563,personal care
18,19,6264,snacks
12,13,5371,pantry
6,7,4365,beverages
0,1,4007,frozen
15,16,3449,dairy eggs
16,17,3085,household
14,15,2092,canned goods
8,9,1858,dry goods pasta
3,4,1684,produce


From the above table, we can observe that personal care seems to be the dept with most number of products. Lets find out the below things:

1) What's the concentration of personal care products/which products in personal care are most ordered? How many aisles do they contain? Is personal care the most ordered department as well? <br />
2) What kind of products do 'Missing' and 'other' department have? What kind of aisles do they have? How much percent of items from those departments are ordered? <br />
3) What exactly is the international section? Does it mean they are imported or just that they have international presence? What products and aisles does the department "international" have? Are they ordered more than non-international items? 

In order to answer all the questions, let us merge product tables, user and order tables.

In [15]:
# Merge products, aisles and departments
products_dept_aisles_df = products.merge(aisles, on='aisle_id').merge(departments, on='department_id')
products_dept_aisles_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [16]:
# Merge order and product_orders

print(order_products.shape)
print(order_products_prior.shape)
print(orders.shape)

user_product_orders = order_products.append(order_products_prior).merge(orders, on='order_id')
user_product_orders.shape

(1384617, 4)
(32434489, 4)
(3421083, 7)


(33819106, 10)

In [17]:
# Merge all tables. Create train dataset

data = user_product_orders.merge(products_dept_aisles_df, on='product_id')
data.shape

(33819106, 15)

In [18]:
data.head()

# To-DO: Check how many products were not ordered at all
# To-Do: are all orders present in order table present in the merged data df. How many are not present?

# FLAW: The data just tells only if an item was ordered or not. It does not tell HOW MANY were ordered/reordered at a time!

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,47901,train,14,4,6,16.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,2993,train,15,0,7,7.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,41425,train,4,3,8,14.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,187205,train,5,4,14,30.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs


**What's the concentration of personal care products/which aisles/products in personal care are most ordered? How many aisles do they contain? Is personal care the most ordered department as well?**

In [0]:
# orders_grouped_by_product1

In [20]:
# The department id of personal care products is 11

# personal_care_products = products_dept_aisles_df[products_dept_aisles_df['department_id'] == 11]
# personal_care_products.groupby(['department', 'aisle']).agg({'department_id': 'count'}).rename(columns={'department_id': 'department_count'}).sort_values(by='department_count', ascending=False)

# Personal care products contain 17 aisles. The aisle Vitamin Supplements has the most number of products in personal care. 
# Are the number of products and concentration of orders directly proportional? - Does not look like. In the cells further dow, most ordered products belong to the produce section. Which makes sense.

# Most ordered products, most ordered department, most ordered aisle

# Where does Vitamin supplements compare?

# orders_grouped_by_product = data.groupby(['product_id', 'product_name', 'aisle', 'department']).agg({'product_id': 'count'}).rename(columns={'product_id': 'product_count'}).sort_values(by='product_count', ascending=False)
# orders_grouped_by_product1 = orders_grouped_by_product.reset_index()
# orders_grouped_by_product1
# The most ordered product is banana! product id = 491291
# Most ordered department seems to be fresh produce. Confirm. TO-DO

# User who has ordered the most

# products[products['product_id'] == ]

# Concentration of orders 

# Is banana the most reordered product? Are the most ordered products the most reordered as well?

# Any time-series analysis possible? - Nope, only day of week and hour of day given
# Find out the duration of the data captured - Not possible, but we can see from "days_since_prior_order" if we can get any signals

# data.sort_values(by='days_since_prior_order', ascending=False)

# From the above, the max is 30 days since prior order. Also some have NaN values. What do they mean? 

#  data.isna().sum()

# The most re-ordered product

most_reordered_products = data.groupby(['product_id', 'product_name', 'aisle', 'department']).agg({'reordered': 'sum'}).rename(columns={'reordered': 'total_reorders'}).sort_values(by='total_reorders', ascending=False)
most_reordered_products

# The top 6 most ordered and most reordered is the same, but from 7th, it changes. 7th most ordered is Large Lemon, but re-ordered is organic whole milk.
# There could be some relationship between most ordered and reordered, but there could be other factors too. TO-DO: FIND OUT!

# Only days_since_prior_order have 2078068 NaN values. Does it mean these were the first orders from new users? 
# Over a period of 1 month, there were 2078068 new/1st orders? Could be more because we don't know how many new users reordered

# TO-DO: Personal care seems to have medicines mixed as well! Is this normal? Shouldn't it be in a pharmacy section?
# TO-DO:  Is the most ordered product likely to be reordered?
# TO-DO: Check distributions and outliers


,,,,total_reorders
product_id,product_name,aisle,department,
24852,Banana,fresh fruits,produce,415166
13176,Bag of Organic Bananas,fresh fruits,produce,329275
21137,Organic Strawberries,fresh fruits,produce,214448
21903,Organic Baby Spinach,packaged vegetables fruits,produce,194939
47209,Organic Hass Avocado,fresh fruits,produce,176173
...,...,...,...,...
45971,12 Inch Taper Candle White,air fresheners candles,household,0
37381,Burts Thick Cut Potato Chips,chips pretzels,snacks,0
15334,Full Size Candy Bars Variety Pack,candy chocolate,snacks,0


In [21]:
# Most ordered and re-ordered aisle
data_grouped_by_aisle = data.groupby(['aisle_id', 'aisle', 'department']).agg({'aisle_id': 'count'}).rename(columns={'aisle_id': 'aisle_count'}).sort_values(by='aisle_count', ascending=False)
data_grouped_by_aisle.reset_index()

,aisle_id,aisle,department,aisle_count
0,24,fresh fruits,produce,3792661
1,83,fresh vegetables,produce,3568630
2,123,packaged vegetables fruits,produce,1843806
3,120,yogurt,dairy eggs,1507583
4,21,packaged cheese,dairy eggs,1021462
...,...,...,...,...
129,44,eye ear care,personal care,9522
130,102,baby bath body care,babies,8909
131,82,baby accessories,babies,8466
132,132,beauty,personal care,6455


In [22]:
data_grouped_by_aisle_reordered = data.groupby(['aisle_id', 'aisle', 'department']).agg({'reordered': 'sum'}).rename(columns={'reordered': 'total_reorders'}).sort_values(by='total_reorders', ascending=False)
data_grouped_by_aisle_reordered.reset_index()

# The most ordered and reordered seem to be the same only for first few aisles. 

,aisle_id,aisle,department,total_reorders
0,24,fresh fruits,produce,2726251
1,83,fresh vegetables,produce,2123540
2,123,packaged vegetables fruits,produce,1178700
3,120,yogurt,dairy eggs,1034957
4,84,milk,dairy eggs,722128
...,...,...,...,...
129,113,frozen juice,frozen,2327
130,118,first aid,personal care,2235
131,44,eye ear care,personal care,2100
132,10,kitchen supplies,household,1874


In [23]:
# Most ordered and re-ordered department
data_grouped_by_dept = data.groupby(['department_id', 'department']).agg({'department_id': 'count'}).rename(columns={'department_id': 'department_count'}).sort_values(by='department_count', ascending=False)
data_grouped_by_dept.reset_index()

,department_id,department,department_count
0,4,produce,9888378
1,16,dairy eggs,5631067
2,19,snacks,3006412
3,7,beverages,2804175
4,1,frozen,2336858
5,13,pantry,1956819
6,3,bakery,1225181
7,15,canned goods,1114857
8,20,deli,1095540
9,9,dry goods pasta,905340


In [24]:
data_grouped_by_dept_reordered = data.groupby(['department_id', 'department']).agg({'reordered': 'sum'}).rename(columns={'reordered': 'total_reorders'}).sort_values(by='total_reorders', ascending=False)
data_grouped_by_dept_reordered.reset_index()
# Likely miscalculation/bug with my code: Is there a row where reordered count is more than ordered count?


# Beverages/snacks - 3rd and 4th positions are swapped. After that few variations in number of ordered and reordered items wrt to dept.
# Bakery and pantry too

,department_id,department,total_reorders
0,4,produce,6432596
1,16,dairy eggs,3773723
2,7,beverages,1832952
3,19,snacks,1727075
4,1,frozen,1268058
5,3,bakery,769880
6,13,pantry,679799
7,20,deli,666231
8,15,canned goods,511317
9,12,meat seafood,420349


In [25]:
# International contains mostly asian/indian and other foods. What kind of products are present in that? 
# Instacart can also get international furniture like the Indian pooja room (many Indian household need one), which is might sell well.
# Check products are in what is in household and aisle 'more household'

personal_care_products = products_dept_aisles_df[products_dept_aisles_df['department_id'] == 17]
personal_care_products.groupby(['department', 'aisle']).agg({'department_id': 'count'}).rename(columns={'department_id': 'department_count'})

products_dept_aisles_df[products_dept_aisles_df['aisle'] == 'more household']

# A stationary section and hardware section could emerge out of "more household" aisle (Check how many were ordered in this category).

# 
household_aisle_products = products_dept_aisles_df[products_dept_aisles_df['aisle'] == 'more household']
# personal_care_products.groupby(['department', 'aisle']).agg({'department_id': 'count'}).rename(columns={'department_id': 'department_count'})

household_aisle_products

,product_id,product_name,aisle_id,department_id,aisle,department
30223,48,"School Glue, Washable, No Run",87,17,more household,household
30224,274,Snack Bags,87,17,more household,household
30225,573,Red Roses,87,17,more household,household
30226,773,Double Control Small Roach Baits Insecticide,87,17,more household,household
30227,964,Toothpicks,87,17,more household,household
...,...,...,...,...,...,...
30526,49055,Lint Traps,87,17,more household,household
30527,49103,Strip & Seal Security Envelopes No. 10,87,17,more household,household
30528,49309,Grill Lighter,87,17,more household,household
30529,49397,CopperTop AAA Alkaline,87,17,more household,household


In [26]:
personal_care_products.groupby(['department', 'aisle']).agg({'department_id': 'count'}).rename(columns={'department_id': 'department_count'})

department_count
department aisle                                       
household  air fresheners candles                   355
           cleaning products                        655
           dish detergents                          204
           food storage                             206
           kitchen supplies                         218
           laundry                                  506
           more household                           308
           paper goods                              322
           plates bowls cups flatware               199
           trash bags liners                        112

In [27]:
products_dept_aisles_df.groupby(['aisle_id', 'aisle', 'department']).agg({'product_id': 'count'}).rename(columns={'product_id': 'product_count'}).sort_values(by='product_count', ascending=False)

,,,product_count
aisle_id,aisle,department,
100,missing,missing,1258
45,candy chocolate,snacks,1246
37,ice cream ice,frozen,1091
47,vitamins supplements,personal care,1038
120,yogurt,dairy eggs,1026
...,...,...,...
113,frozen juice,frozen,47
82,baby accessories,babies,44
32,packaged produce,produce,32


In [28]:
# To-Do: 

# products_dept_aisles_df.groupby(['department_id','department', 'aisle', 'product_name']).agg({'product_id': 'count'}).reset_index()
products_dept_aisles_df[products_dept_aisles_df['department'] == 'other'].groupby(['department_id', 'aisle_id', 'aisle', 'product_name']).agg({'aisle_id': 'count'})

# There seems to be one aisle associated with one department for 'missing'. Coming to products, there seems to be wide variety of categories associated with 'missing'
#To-DO: do a clustering on the 'missing' column names
# For other column, these items seem to be the ones which are placed near the checkout. it could be those.

aisle_id
department_id aisle_id aisle product_name                                             
2             6        other 100% Pure Eucalyptus                                    1
                             100% Pure Moisturizing Apricot Kernel Oil               1
                             100% Pure Peppermint Oil                                1
                             100% Pure Tea Tree Oil                                  1
                             38210 Full Jaw Cuticle Nipper                           1
...                                                                                ...
                             Zero Calorie Tonic Water                                1
                             Zero Calories Apple Sparkling Beverage                  1
                             ZeroMercury 357 Battery - 3 CT                          1
                             ZzzQuil Alcohol-Free Nighttime Liquid Sleep Aid         1
                             for Men Underwear Maximum Absorbency S/M                1

[548 rows x 1 columns]

In [0]:


# To-Do: Apply simple MF
# To-Do: Split into test train or use order_products and order_products_prior as train and test respectively and merge them with all other tables.


In [29]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,47901,train,14,4,6,16.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,2993,train,15,0,7,7.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,41425,train,4,3,8,14.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,187205,train,5,4,14,30.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs


In [30]:
order_products.append(order_products_prior)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [31]:
pd.set_option('display.max_rows', 100000)
products[['product_id', 'aisle_id']].groupby('aisle_id').agg({'product_id': 'count'}).rename(columns={'product_id': 'product_count'}).merge(aisles, on='aisle_id').sort_values('product_count', ascending=False)


,aisle_id,product_count,aisle
99,100,1258,missing
44,45,1246,candy chocolate
36,37,1091,ice cream ice
46,47,1038,vitamins supplements
119,120,1026,yogurt
106,107,989,chips pretzels
93,94,894,tea
20,21,891,packaged cheese
37,38,880,frozen meals
60,61,874,cookies cakes


In [32]:
# Users with most orders. What items have they ordered? Any Outliers?

user_max_orders = data.groupby(['user_id']).agg({'order_id': pd.Series.nunique}).rename(columns={'order_id': 'total_orders_by_user'}).sort_values(by='total_orders_by_user', ascending=False)
# user_id: 102040 (should have 2 orders)
# TO-DO: User to order: one to many or many to many? Obviously one-to-many, but findout.
user_max_orders

# If highest orders placed is 100 and lowest is 3. But there are a lot of items that were not reordered.

,total_orders_by_user
user_id,
112841,100
174555,100
73676,100
176469,100
90584,100
...,...
157043,3
28893,3
28891,3


In [33]:
# user_max_orders.loc[22906]
# Should be 23
# data[data['user_id'] == 22906].groupby(['user_id']).agg({'order_id': pd.Series.nunique})

# Sort users by max products ordered

user_max_products = data.groupby('user_id').agg({'product_id': 'count'}).rename(columns={'product_id': 'user_max_products'}).sort_values(by='user_max_products', ascending=False)
user_max_products
# The mimimum number of products purchased is 3 and max is 3725

,user_max_products
user_id,
201268,3725
129928,3689
164055,3089
176478,2952
186704,2936
...,...
11877,3
38979,3
122081,3


In [34]:
user_max_products.loc[188345]

# Min number of products purchased by a particular user is 3. 
# Min number of orders by a user is also 3

# Find out what products were ordered by a particular user with min purchased order 3 and min purched products = 3

users_with_3_orders  = user_max_orders[user_max_orders['total_orders_by_user'] == 3]
len(users_with_3_orders)

8686

In [35]:
users_with_3_products = user_max_products[user_max_products['user_max_products'] == 3]
print(len(users_with_3_products))

99


In [36]:
users_with_3_products

,user_max_products
user_id,
96613,3
84009,3
27465,3
124615,3
179906,3
134703,3
84062,3
139811,3
93363,3


In [37]:
test = users_with_3_orders[users_with_3_orders.index.isin(users_with_3_products.index)]
len(test)

# All 99 users with 3 orders placed 3 products!

99

In [38]:
data[data['user_id'] == 188345]

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
2066148,1681908,24964,1,0,188345,prior,1,3,13,NaN,Organic Garlic,83,4,fresh vegetables,produce
2069465,1786362,24964,1,1,188345,prior,2,6,11,17.0,Organic Garlic,83,4,fresh vegetables,produce
4344759,2887811,31506,1,0,188345,prior,3,1,19,9.0,Extra Virgin Olive Oil,19,13,oils vinegars,pantry


In [0]:
# TO-DO: Find out how many items were not reordered. Any specific department/aisle?

products_reordered_count = data.groupby(['product_id', 'product_name', 'aisle', 'department']).agg({'reordered': 'sum'})

In [40]:
not_reordered_by_dept = products_reordered_count[products_reordered_count['reordered'] == 0].groupby('department').count().rename(columns={'reordered': 'total_products_not_reordered'}).sort_values(by='total_products_not_reordered', ascending=False).reset_index()
not_reordered_by_dept

,department,total_products_not_reordered
0,personal care,1054
1,pantry,675
2,household,466
3,snacks,290
4,missing,271
5,beverages,221
6,frozen,153
7,dry goods pasta,126
8,canned goods,113
9,international,102


In [41]:
# Department in which every item is reordered
departments[departments['department'].isin(not_reordered_by_dept['department']) == False]

,department_id,department
9,10,bulk


In [42]:
not_reordered_by_aisle = products_reordered_count[products_reordered_count['reordered'] == 0].groupby('aisle').count().rename(columns={'reordered': 'total_products_not_reordered'}).sort_values(by='total_products_not_reordered', ascending=False).reset_index()
not_reordered_by_aisle

,aisle,total_products_not_reordered
0,missing,271
1,vitamins supplements,207
2,spices seasonings,186
3,hair care,136
4,baking supplies decor,110
5,kitchen supplies,103
6,other,94
7,candy chocolate,93
8,more household,90
9,cold flu allergy,87


In [43]:
# Aisles in which every item is reordered
aisles[aisles['aisle'].isin(not_reordered_by_aisle['aisle']) == False]

,aisle_id,aisle
17,18,bulk dried fruits vegetables
31,32,packaged produce
67,68,bulk grains rice dried goods


In [44]:
data[data['user_id'] == 201268].groupby('order_id').count()

,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
order_id,,,,,,,,,,,,,,
53650,64,64,64,64,64,64,64,64,64,64,64,64,64,64
82167,92,92,92,92,92,92,92,92,92,92,92,92,92,92
124205,37,37,37,37,37,37,37,37,37,37,37,37,37,37
181181,55,55,55,55,55,55,55,55,55,55,55,55,55,55
197998,18,18,18,18,18,18,18,18,18,18,18,18,18,18
291943,63,63,63,63,63,63,63,63,63,63,63,63,63,63
423355,4,4,4,4,4,4,4,4,4,4,4,4,4,4
434503,27,27,27,27,27,27,27,27,27,27,27,27,27,27
474348,28,28,28,28,28,28,28,28,28,28,28,28,28,28


In [45]:
# Products that were never ordered
data_group_by_product = data.groupby('product_id').agg({'product_id': 'count'}).rename(columns={'product_id':'product_count'}).reset_index()
products_never_ordered = products[products['product_id'].isin(data_group_by_product['product_id']) == False]
products_never_ordered

,product_id,product_name,aisle_id,department_id
3629,3630,Protein Granola Apple Crisp,57,14
7044,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13
46624,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7


In [46]:
# Products that were never ordered belonged to aisles:
aisles[aisles['aisle_id'].isin(products_never_ordered['aisle_id']) == True]

,aisle_id,aisle
30,31,refrigerated
56,57,granola
87,88,spreads


In [47]:
# Products that were never ordered belonged to departments:
departments[departments['department_id'].isin(products_never_ordered['department_id']) == True]

,department_id,department
6,7,beverages
12,13,pantry
13,14,breakfast


In [0]:
# Users with max products purchased. Any correlation? Ordering in cart, max reorders 

In [0]:
# To-Do: Maximum products seem to be present in aisle 100 which is of category 'missing'. What products are in that? Are they most frequently ordered?

In [50]:
# To-do: do the same with aisle
len(order_products.order_id.unique())

131209

In [0]:
# Check if the missing products and other categories have overlap in products - No, because product to aisle and product to dept is one to many.
# Have missing  products been ordered ever? -  Yeah they have.
# Products that were never ordered belonged to departments beverages, pantry and breakfast

Insights so far:

1. Most ordered/re-ordered item is banana! Fresh produce is the most ordered dept, which makes sense.
2. The departments/aisles with maximum products are NOT the ones most ordered. In fact, its the other way round. For instance the department "personal care" has maximum number of products, but it also has the maximum number of items that were never reordered. The only department in which all items were reordered belonged to "bulk" category.
3. Same with aisle - maximum items that were not reordered belonged to "missing", which also happens to be the aisle containing the most number of products.
4. 3 products out of 49688 were never ordered.
5. Minimum orders placed by users is 3 and surprisingly minimum number of products purchased by users is also 3! For instance, a user with user_id 188345 placed 3 orders, and in every order, only one product was added to the cart. 2 of those orders contained just "Organic Garlic". The problem with the dataset is, it tells if a product was ordered or not. But it does not tell "how many". In this scenario, if Instacart has not set a minimum price for delivery, I think it should.

Now check why #2 and #3 are the way it is. If we can confirm something from the dataset or if we will need outside information.

Also for #5: why do you think this is? what does this tell you or not tell you? is there other data you would ask for if you were able to?





In [75]:
# For #2 and #3, we can check the % of items that were never reordered for each dept and aisle. 
# May be the companies like to keep certain % of such items. Also see why they were not reordered. 
# Any bad reviews from other websites about the product?

# Let us make 6 bins
# % of items reordered 0 times
# % of items reordered 1-5 times
# % of items reordered 5-50 times
# % of items reordered 50-500 times
# % of items reordered 500-5000
# % of items reordered 5000+ times

products_reordered = data.groupby(['product_id', 'product_name', 'aisle', 'department', 'department_id', 'aisle_id']).agg({'reordered': 'sum'})

never_reordered_products = products_reordered[products_reordered['reordered'] == 0]
only_reordered_products = products_reordered[products_reordered['reordered'] > 0]
# greater_than_1 = products_reordered_count[products_reordered_count['reordered'] > 1]
# products_reordered_count.sort_values(by='reordered', ascending=False)

never_reordered_products_by_dept = never_reordered_products.groupby(['department_id', 'department']).agg({'reordered': 'count'}).rename(columns={'reordered': 'not_ordered_products_count'}).reset_index()
only_reordered_products_by_dept = only_reordered_products.groupby(['department_id']).agg({'reordered': 'count'}).reset_index()

ordered_and_unordered_by_dept = never_reordered_products_by_dept.merge(only_reordered_products_by_dept, on='department_id')
ordered_and_unordered_by_dept['percent_unordered'] = (ordered_and_unordered_by_dept['not_ordered_products_count'] / (ordered_and_unordered_by_dept['not_ordered_products_count'] + ordered_and_unordered_by_dept['reordered'])) * 100
ordered_and_unordered_by_dept.sort_values(by='percent_unordered')

,department_id,department,not_ordered_products_count,reordered,percent_unordered
14,16,dairy eggs,94,3355,2.725428
3,4,produce,48,1636,2.850356
12,14,breakfast,35,1079,3.141831
7,8,pets,35,937,3.600823
18,20,deli,48,1274,3.630862
0,1,frozen,153,3854,3.818318
2,3,bakery,61,1455,4.023747
17,19,snacks,290,5974,4.629630
10,12,meat seafood,42,865,4.630650
6,7,beverages,221,4143,5.064161


In [77]:
never_reordered_products_by_aisle = never_reordered_products.groupby(['aisle_id', 'aisle', 'department']).agg({'reordered': 'count'}).rename(columns={'reordered': 'not_ordered_products_count'}).reset_index()
only_reordered_products_by_aisle = only_reordered_products.groupby(['aisle_id']).agg({'reordered': 'count'}).reset_index()

ordered_and_unordered_by_aisle = never_reordered_products_by_aisle.merge(only_reordered_products_by_aisle, on='aisle_id')
ordered_and_unordered_by_aisle['percent_unordered'] = (ordered_and_unordered_by_aisle['not_ordered_products_count'] / (ordered_and_unordered_by_aisle['not_ordered_products_count'] + ordered_and_unordered_by_aisle['reordered'])) * 100
ordered_and_unordered_by_aisle.sort_values(by='percent_unordered', ascending=False)

,aisle_id,aisle,department,not_ordered_products_count,reordered,percent_unordered
9,10,kitchen supplies,household,103,115,47.247706
128,132,beauty,personal care,72,106,40.449438
93,97,baking supplies decor,pantry,110,180,37.931034
83,87,more household,household,90,218,29.220779
78,82,baby accessories,babies,11,33,25.000000
100,104,spices seasonings,pantry,186,611,23.337516
114,118,first aid,personal care,56,184,23.333333
96,100,missing,missing,271,987,21.542130
10,11,cold flu allergy,personal care,87,340,20.374707
105,109,skin care,personal care,49,196,20.000000


For #2 and #3, the % of products that were not reordered is as less as 1% for few categories and almost 50% for other categories. The aisles 'beauty (personal care dept)' and 'kitchen supplies (household dept)' contained 40% and 47% products that were not reordered. They were the top two aisles whose products were not reordered. It makes sense because given the kind of product these aisles have, it takes a while to reorder and in case of kitchen supplies like pans, lids, mixing bowls they might not be reordered at all. Also, the data that Instacart has shared has about one month of orders. For some of the products, like make-up brushes, it might take many months to reorder. I think my hypothesis that departments/aisles with most products had the most unordered products is not fully correct because ordering/reordering boils down to the "type" of product.

RecSys Should take this into consideration while recommending new orders to customer list. its not about only reordered items

For #5, I created an account and tried ordering from instacart's website. We need to have minimum $25 worth products in our cart to be eligible for delivery. So for this usecase, I think the users might have ordered bulk quantity to be eligible for shipping. But this data is from 2017 and it does not tell me the price of the product and the quantity. So I am not able to conclude anything here. I would ask for "price" and "quantity" if I were able to.



In [80]:
products[products['aisle_id'] == 132]

,product_id,product_name,aisle_id,department_id
150,151,Aromatherapaes Stress Reducing Lavender & Cham...,132,11
930,931,Lash Discovery Waterproof Mascara - Very Black,132,11
1041,1042,Perfect Fit Facial Tissue,132,11
1501,1502,Acai Berry Lip Balm,132,11
1549,1550,Mini Slant Pocket Tweezer,132,11
1713,1714,100% Cotton Swabs,132,11
2198,2199,The Best Lip Balm Ever,132,11
2345,2346,Eye Liner Black,132,11
2354,2355,Cushioned Nail Board,132,11
2602,2603,Fashion Lashes - Natural Lashes 110,132,11


In [0]:
# For #5 check Instacart website for minimum orders. 
# The extra data I'd ask for is the price and total quantity ordered.

In [0]:
pd.set_option('display.max_rows', 20000000)
products[products['department_id'] == 21]

In [0]:
# Avg sales per dept

# Train a model to predict which category it should've been (NLP)

products[['aisle_id', 'product_id']].groupby(['aisle_id']).count().merge(aisles, on='aisle_id').sort_values('product_id', ascending=False).rename(columns={'product_id': 'product_count'})

In [0]:

# What does aisle 100 contain? What kind of products? Given that is the aisle which has max number of products. Which department does it belong to?

# To-do: Do some department sell better at given times of the day?
# To-do: Which products stand out in each depts
# To-do: Which departments are concentrated in some products

# Which products were re-ordered the most? 
# Any recurring pattern of similar consecutive products add_to_cart_order (tricky)
# Do orders have a mixture of items or mostly the ones ordered from specific departments?
# Least important products/departments? Why is it least important?

# What kind of products make up personal care? 
# 
# The most re-ordered items.
# Items that were never re-ordered
# Use NLP to see if all products categorization makes sense.

# To-DO: Most busiest aisle
# Any missing aisles
# Where do random/new/trail products go? Are they assigned to a aisle/department once for all or do they goto 'other'/'missing' categories?

# What is it about an item that makes customers order again and again. Is there a re-ordering pattern?

# Set up a training set and testing set

# A person might order the same thing over and over again
# ALS, SVD - baseline

# MF: aisle embeddings, product embeddings
# User product recommender
# User aisle recommender
# User dept recommender
# Or all the above, one gaint dataset.


# TO-DO: Come up with a order basket for user just like how Spotify comes with custom playlists.

References:

https://www.kaggle.com/c/instacart-market-basket-analysis/ <br />
https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2